In [1]:
import pandas as pd

## Preprocess data

In [2]:
a = pd.read_csv("term-deposit-marketing-2020.csv")
a['job'].replace(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], inplace=True)
a['marital'].replace(['married', 'single', 'divorced'],
                        [0, 1, 2], inplace=True)
a['education'].replace(['tertiary', 'secondary', 'unknown', 'primary'],
                        [0, 1, 2, 3], inplace=True)
a['default'].replace(['no', 'yes'],
                        [0, 1], inplace=True)
a['housing'].replace(['no', 'yes'],
                        [0, 1], inplace=True)
a['loan'].replace(['no', 'yes'],
                        [0, 1], inplace=True)
a['contact'].replace(['unknown', 'cellular', 'telephone'],
                        [0, 1, 2], inplace=True)
a['month'].replace(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'jan', 'feb',
       'mar', 'apr'],
                        [5, 6, 7, 8, 10, 11, 12, 1, 2, 3, 4], inplace=True)
a['y'].replace(["no", "yes"],
                        [0,1], inplace=True)
X = a.drop(["y"], axis=1)
y = a["y"]

In [3]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
d = {}
for i, (train_index, test_index) in enumerate(kf.split(X)):
    d["train{0}".format(i)] = train_index
    d["test{0}".format(i)] = test_index
train2 = X.loc[d["train2"]]
test2 = X.loc[d["test2"]]
y_train2 = y[d["train2"]]
y_test2 = y[d["test2"]]

In [4]:
y_train2 = y_train2.reset_index(drop=True)

In [6]:
ind = []
for i in range(32000):
    if y_train2[i] == 1:
        ind.append(i)

In [7]:
new_X = train2.iloc[ind]

In [8]:
ind1 = []
for i in range(32000):
    if y_train2[i] == 0:
        ind1.append(i)

In [9]:
new_X1 = train2.iloc[ind1]

In [10]:
len(ind)

2457

In [11]:
new_X2 = new_X1.sample(n=7543, replace=False, random_state=0)

In [12]:
new_X3 = pd.concat([new_X,new_X2])
y_train = []
for i in range(2457):
    y_train.append(1)
for i in range(7543):
    y_train.append(0)

## Dropped day and month

In [13]:
new_X3 = new_X3.drop(columns = ["day", "month"])

In [15]:
test2 = test2.drop(columns = ["day", "month"])

In [16]:
from catboost import CatBoostClassifier, metrics

In [17]:
model = CatBoostClassifier(
    class_weights = [3, 1],
    custom_loss=[metrics.CrossEntropy()],
    random_seed=42,
    logging_level='Silent'
)
model.fit(new_X3, y_train)

In [19]:
predictions = model.predict(test2)

In [20]:
from sklearn.metrics import f1_score
f1_score(y_test2, predictions)

0.5601703940362087

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test2, predictions)

array([[7324,  237],
       [ 176,  263]])